<a href="https://colab.research.google.com/github/chengyang122/mutitaskNAM/blob/main/MutiTaskClassificationTutoral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/chengyang122/mutitaskNAM.git

fatal: destination path 'mutitaskNAM' already exists and is not an empty directory.


In [2]:
cd mutitaskNAM

/content/mutitaskNAM


In [3]:
import os
import tqdm
import copy
import random
import logging
from absl import app
from absl import flags
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
from nam.metrics import *
import nam.data_utils
from nam.model import *

In [8]:
#for muti task classification 200 classes 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features = pd.read_csv('features.csv', index_col=0)
target = pd.read_csv('target.csv', index_col=0, dtype = float)
s = target['0']
oneHotTarget = pd.get_dummies(s)
x_train = features.to_numpy()[:4000]
y_train = oneHotTarget.to_numpy()[:4000]
x_val = features.to_numpy()[4000:]
y_val = oneHotTarget.to_numpy()[4000:]
train_dataset = TensorDataset(torch.tensor(x_train), torch.tensor(y_train))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(torch.tensor(x_val), torch.tensor(y_val))
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

criterion = penalized_cross_entropy_MutiTask

In [7]:
#for 2 class classification 
train, (x_test, y_test) = nam.data_utils.create_test_train_fold(dataset='BreastCancer',
                                                                id_fold=1,
                                                                n_folds=5,
                                                                n_splits=3,
                                                                regression=False)
(x_train, y_train), (x_validate, y_validate) = next(train)
train_dataset = TensorDataset(torch.tensor(x_train), torch.tensor(y_train))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = TensorDataset(torch.tensor(x_validate), torch.tensor(y_validate))
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)
(x_train, y_train), (x_validate, y_validate) = next(train)

criterion = penalized_cross_entropy

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str_']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str_']. An error will be raised in 1.2.
  FutureWarning,


In [9]:
model = NeuralAdditiveModel(
        input_size=x_train.shape[-1],
        # feature size, 0 is sample and 1 is the feature, this is one iter of torch dataloader
        output_size=1 if len(y_train.shape)==1 else y_train.shape[-1],
        shallow_units=nam.data_utils.calculate_n_units(x_train, 1000, 2),
        # for feature network, it is changing with data and I am not sure why
        hidden_units=list(map(int, [])),  # for feature network
        shallow_layer=ExULayer,  # special operational layer designed for this model
        hidden_layer=ExULayer,
        hidden_dropout=0.3,
        feature_dropout=0.0).to(device)

In [10]:
regression = False
optimizer = torch.optim.AdamW(model.parameters(),
                              lr=1e-3,
                              weight_decay=0.0)

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=0.995, step_size=1)


In [11]:
def train_one_epoch(model, criterion, optimizer, data_loader, device):
    pbar = tqdm.tqdm(enumerate(data_loader, start=1), total=len(data_loader))
    total_loss = 0
    for i, (x, y) in pbar:
        x, y = x.to(device), y.to(device)
        logits, fnns_out = model.forward(x)
        loss = criterion(logits, y, fnns_out, feature_penalty=0.0)
        total_loss -= (total_loss / i) - (loss.item() / i)
        model.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.set_description(f"train | loss = {total_loss:.5f}")
    return total_loss

In [12]:
def evaluate(model, data_loader, device):
    total_score = 0
    metric = None
    for i, (x, y) in enumerate(data_loader, start=1):
        x, y = x.to(device), y.to(device)
        logits, fnns_out = model.forward(x)
        metric, score = calculate_metric(logits, y, regression=False)
        total_score -= (total_score / i) - (score / i)
    return metric, total_score

In [13]:
val_scores = []
best_validation_score, best_weights = 0, None
n_tries = 60
for epoch in range(10):
    model = model.train()
    total_loss = train_one_epoch(model, criterion, optimizer, train_loader, device)
    logging.info(f"epoch {epoch} | train | {total_loss}")

    scheduler.step()

    model = model.eval()
    metric, val_score = evaluate(model, val_loader, device)
    metric, train_score = evaluate(model, train_loader, device)
    print(f"epoch {epoch} | validate | {metric}={val_score}")
    print(f"epoch {epoch} | train | {metric}={train_score}")
    # early stopping
    if val_score <= best_validation_score and n_tries > 0:
        n_tries -= 1
        continue
    elif val_score <= best_validation_score:
        logging.info(f"early stopping at epoch {epoch}")
        break
    best_validation_score = val_score
    best_weights = copy.deepcopy(model.state_dict())
    val_scores.append(val_score)
model.load_state_dict(best_weights)

train | loss = 5.19127: 100%|██████████| 63/63 [00:05<00:00, 11.03it/s]


epoch 0 | validate | accuracy=0.002097355703321787
epoch 0 | train | accuracy=0.0024231150321337206


train | loss = 3.11626: 100%|██████████| 63/63 [00:05<00:00, 11.10it/s]


epoch 1 | validate | accuracy=0.004162087821616577
epoch 1 | train | accuracy=0.0043328371941156315


train | loss = 2.02093: 100%|██████████| 63/63 [00:05<00:00, 11.16it/s]


...

train | loss = 0.42841: 100%|██████████| 63/63 [00:05<00:00, 11.46it/s]


epoch 8 | validate | accuracy=0.004830013674039108
epoch 8 | train | accuracy=0.004858630916310683


train | loss = 0.36147: 100%|██████████| 63/63 [00:05<00:00, 11.06it/s]


epoch 9 | validate | accuracy=0.004915865353093698
epoch 9 | train | accuracy=0.004894593227950354


<All keys matched successfully>